In [6]:
import sqlite3
import os

def connect():
    database_path = os.environ["CHESS_ANALYSIS_DB"]
    connection = sqlite3.connect(database_path)
    return connection

In [7]:
from pathlib import Path
import sqlparse

def load_sql(sql_file, placeholder_values = None, show_query = False):
    # Read the query file
    query_file_path = Path("sql") / sql_file
    with open(query_file_path.absolute(), "r") as f:
        query = f.read().replace("\n", " ")

    # Replace placeholders with real values
    if placeholder_values:
        for key, value in placeholder_values.items():
            query = query.replace(f"${key}", value)

    # Show a pretty-printed form of the query
    if show_query:
        print(sqlparse.format(query, reindent=True, keyword_case="upper"))
    return query


In [8]:
import pandas as pd

def load_player_game_ids(connection, name):
    query = load_sql("load_player_game_ids.sql", { "NAME" : name })
    df = pd.read_sql_query(query, connection)
    return df

In [9]:
def load_analysis(connection, game_id, engine):
    # Construct the query
    query = load_sql("load_analysis.sql", {
        "GAME_ID" : str(game_id),
        "ENGINE": engine
    })

    # Read the data
    df = pd.read_sql_query(query, connection)

    # Prepend the move and player indicator
    pos = df.columns.get_loc("halfmove")
    df.insert(pos, "player", df.index.map(lambda i: WHITE if ((1 + i) % 2) > 0 else BLACK))

    pos = df.columns.get_loc("halfmove")
    df.insert(pos, "move", 1 + df.index // 2)

    return df

In [ ]:
from datetime import datetime as dt

def load_game_metadata(connection, game_id):
    query = load_sql("load_metadata.sql", { "GAME_ID" : str(game_id) })
    df = pd.read_sql_query(query, connection)

    # See if the date row is present and, if so, attempt to parse the date as a datetime and write it back
    # If it's not present, or it doesn't parse, make sure the Date row is present with a value of None
    date_row = df.loc[df["name"] == "Date", "value"]
    if not date_row.empty:
        try:
            date_dt = dt.strptime(date_row.iloc[0], "%Y.%m.%d").date()
        except ValueError:
            date_dt = None
        df.loc[df["name"] == "Date", "value"] = date_dt
    else:
        df.loc[len(df)] = {"name": "Date", "value": None}

    return df